## Парсинг сайта Banki.ru

Использую selenium webdriver, т.к. на первый взгляд bank.ru используют AJAX, и просто так scrapy завести не удалось. Selenium для меня достаточно знакомый инструмент, поэтому времени он занял немного

In [15]:
from selenium import webdriver
from bs4 import BeautifulSoup
import numpy as np
from tqdm import tqdm, tqdm_notebook
import time
import requests
import pandas as pd

from multiprocessing.dummy import Pool as ThreadPool

In [16]:
class BankParser():
    def __init__(self, raw_page="https://www.banki.ru/services/responses/list/?is_countable=on&page={}"):
        self.browser = webdriver.Chrome("./chromedriver")
        self.raw_page = raw_page
        self.feedback_urls = []
    
    def parse_one_page(self, url):
        self.browser.get(url)
        soup = BeautifulSoup(self.browser.page_source)
        hrefs = soup.find_all(text="Весь отзыв", href=True)
        hrefs_pretty = list(map(lambda href: "https://www.banki.ru" + href.attrs["href"], hrefs))
        time.sleep(np.random.uniform(low=4, high=8))
        return hrefs_pretty
    
    def parse(self, pages_id):
        pages_id = list(pages_id)
        for page_id in tqdm(pages_id):
            self.feedback_urls.extend(self.parse_one_page(self.raw_page.format(page_id)))
        
    def dump_feedback_urls(self, name):
        self.feedback_urls = np.array(self.feedback_urls)
        np.save(name, self.feedback_urls)

In [17]:
bank_parser = BankParser()

In [18]:
bank_parser.parse(range(351, 450))


100%|██████████| 99/99 [12:39<00:00,  7.30s/it]


In [19]:
bank_parser.dump_feedback_urls("./data/url_feedbacks_351_450.npy")

In [22]:
class FeedbackParser():
    def __init__(self, path_to_feedback_urls):
        self.path_to_feedback_urls = path_to_feedback_urls
        self.feedbacks = []
    
    def load_feedback_urls(self):
        self.feedback_urls = []
        for path_to_feedback in self.path_to_feedback_urls:
            self.feedback_urls.extend(list(np.load(path_to_feedback)))
        
        print("cnt feedbacks = {}".format(len(self.feedback_urls)))
        
    def parse_one(self, url):
        request = requests.get(url)
        soup = BeautifulSoup(request.text)
        score = soup.find(itemprop="rating")
        text = soup.find(itemprop="description").text.strip()
        return text, int(score.text.strip())
        
    def parse(self):
        for feedback_url in tqdm(self.feedback_urls):
            text, score = self.parse_one(feedback_url)
            self.feedbacks.append([text, score])
            
    def dump_feedback(self, name):
        data = pd.DataFrame(data=self.feedbacks, columns=["text", "score"])
        data.to_csv(name)

In [23]:
feedback_parser = FeedbackParser(["./data/url_feedbacks_351_450.npy"])
feedback_parser.load_feedback_urls()
feedback_parser.parse()



  0%|          | 0/2465 [00:00<?, ?it/s]

cnt feedbacks = 2465




  0%|          | 1/2465 [00:00<24:04,  1.71it/s]

  0%|          | 2/2465 [00:01<23:44,  1.73it/s]

  0%|          | 3/2465 [00:01<22:34,  1.82it/s]

  0%|          | 4/2465 [00:02<21:29,  1.91it/s]

  0%|          | 5/2465 [00:02<20:58,  1.95it/s]

  0%|          | 6/2465 [00:03<21:33,  1.90it/s]

  0%|          | 7/2465 [00:03<20:40,  1.98it/s]

  0%|          | 8/2465 [00:04<21:00,  1.95it/s]

  0%|          | 9/2465 [00:04<20:52,  1.96it/s]

  0%|          | 10/2465 [00:05<20:59,  1.95it/s]

  0%|          | 11/2465 [00:05<20:54,  1.96it/s]

  0%|          | 12/2465 [00:06<20:35,  1.99it/s]

  1%|          | 13/2465 [00:06<22:33,  1.81it/s]

  1%|          | 14/2465 [00:07<21:58,  1.86it/s]

  1%|          | 15/2465 [00:07<21:34,  1.89it/s]

  1%|          | 16/2465 [00:08<21:30,  1.90it/s]

  1%|          | 17/2465 [00:08<20:55,  1.95it/s]

  1%|          | 18/2465 [00:09<21:35,  1.89it/s]

  1%|          | 19/2465 [00:09<21:29,  1.90it/s]

  1%|          | 20/2465 [00:10<22:17,

  6%|▌         | 144/2465 [01:27<22:14,  1.74it/s]

  6%|▌         | 145/2465 [01:28<25:08,  1.54it/s]

  6%|▌         | 146/2465 [01:28<27:42,  1.39it/s]

  6%|▌         | 147/2465 [01:29<25:31,  1.51it/s]

  6%|▌         | 148/2465 [01:30<25:47,  1.50it/s]

  6%|▌         | 149/2465 [01:30<23:36,  1.63it/s]

  6%|▌         | 150/2465 [01:31<22:32,  1.71it/s]

  6%|▌         | 151/2465 [01:31<23:37,  1.63it/s]

  6%|▌         | 152/2465 [01:32<23:44,  1.62it/s]

  6%|▌         | 153/2465 [01:33<24:14,  1.59it/s]

  6%|▌         | 154/2465 [01:33<23:00,  1.67it/s]

  6%|▋         | 155/2465 [01:34<23:54,  1.61it/s]

  6%|▋         | 156/2465 [01:34<22:37,  1.70it/s]

  6%|▋         | 157/2465 [01:35<22:34,  1.70it/s]

  6%|▋         | 158/2465 [01:36<23:25,  1.64it/s]

  6%|▋         | 159/2465 [01:36<24:36,  1.56it/s]

  6%|▋         | 160/2465 [01:37<23:45,  1.62it/s]

  7%|▋         | 161/2465 [01:37<23:04,  1.66it/s]

  7%|▋         | 162/2465 [01:38<26:02,  1.47it/s]

  7%|▋      

 12%|█▏        | 285/2465 [02:55<23:38,  1.54it/s]

 12%|█▏        | 286/2465 [02:55<22:37,  1.61it/s]

 12%|█▏        | 287/2465 [02:56<21:44,  1.67it/s]

 12%|█▏        | 288/2465 [02:57<24:03,  1.51it/s]

 12%|█▏        | 289/2465 [02:57<22:06,  1.64it/s]

 12%|█▏        | 290/2465 [02:58<21:09,  1.71it/s]

 12%|█▏        | 291/2465 [02:58<22:00,  1.65it/s]

 12%|█▏        | 292/2465 [02:59<23:18,  1.55it/s]

 12%|█▏        | 293/2465 [03:00<22:54,  1.58it/s]

 12%|█▏        | 294/2465 [03:00<22:50,  1.58it/s]

 12%|█▏        | 295/2465 [03:01<23:30,  1.54it/s]

 12%|█▏        | 296/2465 [03:02<23:13,  1.56it/s]

 12%|█▏        | 297/2465 [03:02<21:36,  1.67it/s]

 12%|█▏        | 298/2465 [03:03<20:48,  1.74it/s]

 12%|█▏        | 299/2465 [03:03<20:13,  1.79it/s]

 12%|█▏        | 300/2465 [03:04<21:14,  1.70it/s]

 12%|█▏        | 301/2465 [03:04<22:23,  1.61it/s]

 12%|█▏        | 302/2465 [03:05<21:18,  1.69it/s]

 12%|█▏        | 303/2465 [03:06<23:34,  1.53it/s]

 12%|█▏     

...

 74%|███████▍  | 1819/2465 [18:05<05:51,  1.84it/s]

 74%|███████▍  | 1820/2465 [18:05<05:37,  1.91it/s]

 74%|███████▍  | 1821/2465 [18:06<05:36,  1.92it/s]

 74%|███████▍  | 1822/2465 [18:06<05:25,  1.98it/s]

 74%|███████▍  | 1823/2465 [18:07<05:29,  1.95it/s]

 74%|███████▍  | 1824/2465 [18:07<05:28,  1.95it/s]

 74%|███████▍  | 1825/2465 [18:08<05:16,  2.02it/s]

 74%|███████▍  | 1826/2465 [18:09<06:35,  1.62it/s]

 74%|███████▍  | 1827/2465 [18:09<06:11,  1.72it/s]

 74%|███████▍  | 1828/2465 [18:10<06:21,  1.67it/s]

 74%|███████▍  | 1829/2465 [18:10<06:11,  1.71it/s]

 74%|███████▍  | 1830/2465 [18:11<05:45,  1.84it/s]

 74%|███████▍  | 1831/2465 [18:11<06:02,  1.75it/s]

 74%|███████▍  | 1832/2465 [18:12<05:48,  1.82it/s]

 74%|███████▍  | 1833/2465 [18:12<05:42,  1.85it/s]

 74%|███████▍  | 1834/2465 [18:13<06:02,  1.74it/s]

 74%|███████▍  | 1835/2465 [18:14<05:59,  1.75it/s]

 74%|███████▍  | 1836/2465 [18:14<06:40,  1.57it/s]

 75%|███████▍  | 1837/2465 [18:15<06:17,  1.66

 79%|███████▉  | 1957/2465 [19:25<04:40,  1.81it/s]

 79%|███████▉  | 1958/2465 [19:25<04:30,  1.87it/s]

 79%|███████▉  | 1959/2465 [19:26<04:20,  1.94it/s]

 80%|███████▉  | 1960/2465 [19:26<04:15,  1.98it/s]

 80%|███████▉  | 1961/2465 [19:27<04:52,  1.73it/s]

 80%|███████▉  | 1962/2465 [19:28<04:47,  1.75it/s]

 80%|███████▉  | 1963/2465 [19:28<04:54,  1.70it/s]

 80%|███████▉  | 1964/2465 [19:29<04:43,  1.77it/s]

 80%|███████▉  | 1965/2465 [19:29<04:50,  1.72it/s]

 80%|███████▉  | 1966/2465 [19:30<04:41,  1.77it/s]

 80%|███████▉  | 1967/2465 [19:30<04:29,  1.85it/s]

 80%|███████▉  | 1968/2465 [19:31<04:15,  1.94it/s]

 80%|███████▉  | 1969/2465 [19:31<04:40,  1.77it/s]

 80%|███████▉  | 1970/2465 [19:32<04:23,  1.88it/s]

 80%|███████▉  | 1971/2465 [19:33<04:42,  1.75it/s]

 80%|████████  | 1972/2465 [19:33<04:27,  1.84it/s]

 80%|████████  | 1973/2465 [19:34<04:14,  1.93it/s]

 80%|████████  | 1974/2465 [19:34<04:58,  1.65it/s]

 80%|████████  | 1975/2465 [19:35<04:51,  1.68

 85%|████████▍ | 2095/2465 [21:10<03:37,  1.70it/s]

 85%|████████▌ | 2096/2465 [21:10<03:27,  1.78it/s]

 85%|████████▌ | 2097/2465 [21:11<03:37,  1.69it/s]

 85%|████████▌ | 2098/2465 [21:11<03:27,  1.77it/s]

 85%|████████▌ | 2099/2465 [21:12<03:35,  1.70it/s]

 85%|████████▌ | 2100/2465 [21:12<03:24,  1.79it/s]

 85%|████████▌ | 2101/2465 [21:13<03:15,  1.87it/s]

 85%|████████▌ | 2102/2465 [21:13<03:10,  1.91it/s]

 85%|████████▌ | 2103/2465 [21:14<03:19,  1.81it/s]

 85%|████████▌ | 2104/2465 [21:15<03:27,  1.74it/s]

 85%|████████▌ | 2105/2465 [21:15<03:39,  1.64it/s]

 85%|████████▌ | 2106/2465 [21:16<03:32,  1.69it/s]

 85%|████████▌ | 2107/2465 [21:16<03:16,  1.82it/s]

 86%|████████▌ | 2108/2465 [21:17<03:00,  1.98it/s]

 86%|████████▌ | 2109/2465 [21:17<03:00,  1.98it/s]

 86%|████████▌ | 2110/2465 [21:18<02:55,  2.02it/s]

 86%|████████▌ | 2111/2465 [21:18<02:54,  2.03it/s]

 86%|████████▌ | 2112/2465 [21:19<02:58,  1.98it/s]

 86%|████████▌ | 2113/2465 [21:19<02:53,  2.03

 91%|█████████ | 2233/2465 [22:27<02:01,  1.92it/s]

 91%|█████████ | 2234/2465 [22:28<01:59,  1.93it/s]

 91%|█████████ | 2235/2465 [22:28<01:56,  1.97it/s]

 91%|█████████ | 2236/2465 [22:29<01:55,  1.98it/s]

 91%|█████████ | 2237/2465 [22:29<01:54,  1.98it/s]

 91%|█████████ | 2238/2465 [22:30<01:56,  1.94it/s]

 91%|█████████ | 2239/2465 [22:30<02:01,  1.86it/s]

 91%|█████████ | 2240/2465 [22:31<02:07,  1.76it/s]

 91%|█████████ | 2241/2465 [22:31<02:01,  1.84it/s]

 91%|█████████ | 2242/2465 [22:32<02:01,  1.84it/s]

 91%|█████████ | 2243/2465 [22:33<02:08,  1.73it/s]

 91%|█████████ | 2244/2465 [22:33<02:09,  1.71it/s]

 91%|█████████ | 2245/2465 [22:34<02:13,  1.65it/s]

 91%|█████████ | 2246/2465 [22:35<02:31,  1.44it/s]

 91%|█████████ | 2247/2465 [22:35<02:35,  1.40it/s]

 91%|█████████ | 2248/2465 [22:36<02:18,  1.56it/s]

 91%|█████████ | 2249/2465 [22:36<02:11,  1.65it/s]

 91%|█████████▏| 2250/2465 [22:37<02:02,  1.75it/s]

 91%|█████████▏| 2251/2465 [22:38<02:01,  1.76

 96%|█████████▌| 2371/2465 [23:47<00:52,  1.79it/s]

 96%|█████████▌| 2372/2465 [23:48<00:50,  1.83it/s]

 96%|█████████▋| 2373/2465 [23:48<00:48,  1.88it/s]

 96%|█████████▋| 2374/2465 [23:49<00:48,  1.86it/s]

 96%|█████████▋| 2375/2465 [23:49<00:47,  1.88it/s]

 96%|█████████▋| 2376/2465 [23:50<00:46,  1.90it/s]

 96%|█████████▋| 2377/2465 [23:50<00:45,  1.92it/s]

 96%|█████████▋| 2378/2465 [23:51<00:43,  1.98it/s]

 97%|█████████▋| 2379/2465 [23:51<00:42,  2.03it/s]

 97%|█████████▋| 2380/2465 [23:52<00:42,  1.98it/s]

 97%|█████████▋| 2381/2465 [23:52<00:43,  1.93it/s]

 97%|█████████▋| 2382/2465 [23:53<00:43,  1.93it/s]

 97%|█████████▋| 2383/2465 [23:53<00:43,  1.87it/s]

 97%|█████████▋| 2384/2465 [23:54<00:43,  1.86it/s]

 97%|█████████▋| 2385/2465 [23:55<00:44,  1.78it/s]

 97%|█████████▋| 2386/2465 [23:55<00:43,  1.82it/s]

 97%|█████████▋| 2387/2465 [23:56<00:42,  1.85it/s]

 97%|█████████▋| 2388/2465 [23:56<00:43,  1.78it/s]

 97%|█████████▋| 2389/2465 [23:57<00:41,  1.84

In [24]:
feedback_parser.dump_feedback("./data/feedbacks_351_450.csv")

In [25]:
len(feedback_parser.feedbacks)

2465